<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/snli_figure2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!rm -rf input-marginalization
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
save_dir = "/content/gdrive/My Drive/NLP/imarg"

Mounted at /content/gdrive


In [5]:
%%capture
import sys
sys.path.append("input-marginalization")

from metrics import input_marginalization, continuous_colored_sentence
import torch
from transformers import BertForMaskedLM, BertTokenizer

In [6]:
%%capture
mlm = BertForMaskedLM.from_pretrained('bert-base-uncased').cuda()

In [7]:
#bert = torch.load(f"{save_dir}/bert_sst2.pt")
#cnn = torch.load(f"{save_dir}/cnn_sst2.pt")
lstm = torch.load(f"{save_dir}/lstm_snli.pt")

In [8]:
lstm_sentences = [
  ("pacino is brilliant as the sleep - deprived dormer , his increasing weariness as much existential as it is physical .", 1),
  ("an important movie , a reminder of the power of file to move us and to make us examine our values .", 1),
  ("unflinchingly bleak and desperate", 0),
  ("i am sorry that i was unable to get the full brunt of this comedy .", 0),
]
bert_sentences = [
  ("it ' s a lovely film with lovely performances by buy and accorsi .", 1),
  ("more romantic , emotional and ultimately more satisfying than the teary - eyed original .", 1),
  ("it ' s a bit disappointing that it only manages to be decent instead of dead brilliant .", 0),
  ("suffers from the lack of a compelling or comprehensible narrative .", 0),   
]
cnn_sentences = [
  (". . . very funny , very enjoyable . . .", 1),
  ("lookin ' for sin , american - style ?", 0),
]

snli_lstm_sentences = [
  (('Two women are embracing while holding to go packages.', 'The sisters are hugging goodbye while holding to go packages after just eating lunch.'), 2),
  (('bicyclists waiting at an intersection.', 'the bicyclists are dead.'), 0)
]

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [14]:
for sentences, label in snli_lstm_sentences:
  (s1, a1), (s2, a2) = input_marginalization(lstm, sentences, mlm, target_label=label, dataset='snli')
  
  #print(att_scores)
  print(f'pre: {continuous_colored_sentence(s1, a1.unsqueeze(0), pretok=True, verbose=False)} | hypo: {continuous_colored_sentence(s2, a2.unsqueeze(0), pretok=True, verbose=False)}')

pre:  two women are embracing while holding to go packages . | hypo:  sisters are hugging goodbye while holding to go packages after just eating lunch .
pre:  bicyclists waiting at an intersection . | hypo:  bicyclists are dead .


In [ ]:
tokenizer.decode(s2[0])

In [ ]:
# try:
#   for sentence, label in bert_sentences:
#     att_scores = input_marginalization(bert, sentence, mlm, target_label=label)
#     color_sentence(sentence, att_scores)
# except KeyboardInterrupt:
#   print("Graceful Exit")

In [ ]:
# for sentence, label in cnn_sentences:
#   att_scores = input_marginalization(cnn, sentence, mlm, target_label=label)
#   color_sentence(sentence, att_scores)

In [ ]:
print(a2.shape)